In [19]:
import numpy as np
import pandas as pd
import os, json, math, time
from tqdm.notebook import tqdm_notebook
import tmdbsimple as tmdb

In [20]:
# Install tmdbsimple
!pip install tmdbsimple

In [21]:
# Install tmdbsimple
!pip install tqdm

In [22]:
with open('/Users/qfu88/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
login.keys()

dict_keys(['api-key'])

In [23]:
tmdb.API_KEY =  login['api-key']

In [24]:
#define a function to add the certification with the other key information.
def get_movie_certification(imdb_id):
    movie = tmdb.Movies(imdb_id)
    info = movie.info()
    releases = movie.releases()
    for c in releases['countries']:
        if c['iso_3166_1'] == 'US':
            info['certification'] = c['certification']
    return info

In [25]:
#define the write_json function
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [26]:
#designate a folder
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_rating.csv.gz',
 'tmdb_api.json',
 'tmdb_api_results_2001.json']

In [27]:
#load in the title basics data
basics = pd.read_csv('Data/title_basics.csv.gz')

In [28]:
# create required lists for the loop
## define a list of the year to extract from the API
YEARS_TO_GET = [2000,2001]

In [29]:
#define an errors list 
errors = [ ]

In [30]:
## set up progress bar
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    time.sleep(0.02)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

In [31]:
#Defining the JSON file to store results for year
JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
# Check if file exists
file_exists = os.path.isfile(JSON_FILE)
if file_exists == False:
         with open(JSON_FILE,'w') as f:
                json.dump([{'imdb_id':0}],f)

In [32]:
#define the IDs to call
#Saving new year as the current df
df = basics.loc[ basics['startYear']==YEAR].copy()
# saving movie ids to list
movie_ids = df['tconst'].copy()

In [33]:
#Check for and remove any previously downloaded Movie id's
## Load existing data from json into a dataframe called "previous_df"
previous_df = pd.read_json(JSON_FILE)
# filter out any ids that are already in the JSON_FILE
movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

In [39]:
    #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
            # Retrieve then data for the movie id
            temp = get_movie_certification(imdb_id)  
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])



Movies from 2001:   0%|          | 0/2840 [00:00<?, ?it/s]